In [ ]:
import functools

def catch_exceptions(job_func, cancel_on_failure=False):
    @functools.wraps(job_func)
    def wrapper(*args, **kwargs):
        try:
            return job_func(*args, **kwargs)
        except:
            import traceback
            print(traceback.format_exc())
            if cancel_on_failure:
                return schedule.CancelJob
    return wrapper

@catch_exceptions(cancel_on_failure=True)
def bad_task():
    return 1 / 0

schedule.every(5).seconds.do(bad_task)

In [ ]:
import threading
import time
import schedule
import arrow

def job():
    date = arrow.now().format('mm:ss')
    print("I'm running on job",date)

def job2():
    date = arrow.now().format('mm:ss')
    print("I'm running on job2",date)

def run_threaded(job_func):
    job_thread = threading.Thread(target=job_func)
    job_thread.start()


schedule.every(10).seconds.do(run_threaded, job)
schedule.every(10).seconds.do(run_threaded, job2)



while 1:
    schedule.run_pending()
    time.sleep(1)

("I'm running on job", '24:47')
("I'm running on job2", '24:47')
("I'm running on job", '24:57')
("I'm running on job2", '24:57')
("I'm running on job", '25:07')
("I'm running on job2", '25:07')


# 一、 数据获取余票数据

# 1.几行简单代码获取某区间某日所有余票数据

## 1.1读取SHANGHAI-SUZHOU单个页面数据

首先要分析网站，怎么样才能获取数据

In [379]:
import requests
import json

In [380]:
url = 'https://kyfw.12306.cn/otn/lcxxcx/query?purpose_codes=ADULT&queryDate=2016-12-29&from_station=SHH&to_station=SZH'

In [381]:
js = requests.get(url).text

SSLError: [Errno 1] _ssl.c:507: error:14090086:SSL routines:SSL3_GET_SERVER_CERTIFICATE:certificate verify failed

https://promotion.aliyun.com/ntms/act/globalhttps.html?spm=5176.8142029.418686.5.yBwhSy<br>
HTTPS在HTTP的基础上加入了SSL/TLS协议，依靠SSL证书来验证服务器的身份，并为客户端和服务器端之间建立“SSL加密通道”，确保用户数据在传输过程中处于加密状态，同时防止服务器被钓鱼网站假冒。

In [382]:
js = requests.get(url,verify=False).text

In [383]:
import warnings
warnings.filterwarnings("ignore")

In [384]:
js = requests.get(url,verify=False).text

In [385]:
js2 = json.loads(js)

In [386]:
data = js2['data']['datas']

In [387]:
data

[{u'arrive_time': u'05:31',
  u'canWebBuy': u'Y',
  u'control_day': 29,
  u'control_train_day': u'20300303',
  u'controlled_train_flag': u'0',
  u'controlled_train_message': u'\u6b63\u5e38\u8f66\u6b21\uff0c\u4e0d\u53d7\u63a7',
  u'day_difference': u'0',
  u'end_city_code': u'0705',
  u'end_province_code': u'07',
  u'end_station_name': u'\u5357\u4eac',
  u'end_station_telecode': u'NJH',
  u'from_station_name': u'\u4e0a\u6d77\u5357',
  u'from_station_no': u'14',
  u'from_station_telecode': u'SNH',
  u'gg_num': u'--',
  u'gr_num': u'--',
  u'is_support_card': u'0',
  u'lishi': u'01:17',
  u'lishiValue': u'77',
  u'location_code': u'G1',
  u'note': u'',
  u'qt_num': u'--',
  u'rw_num': u'11',
  u'rz_num': u'--',
  u'sale_time': u'1530',
  u'seat_feature': u'W3431333',
  u'seat_types': u'1413',
  u'start_city_code': u'1004',
  u'start_province_code': u'10',
  u'start_station_name': u'\u798f\u5dde',
  u'start_station_telecode': u'FZS',
  u'start_time': u'04:14',
  u'start_train_date': u'2016

## 1.2将数据读入pandas并进行处理后保存

In [388]:
import pandas as pd
import numpy as np
df = pd.DataFrame(data)
df.head()

,arrive_time,canWebBuy,control_day,control_train_day,controlled_train_flag,controlled_train_message,day_difference,end_city_code,end_province_code,end_station_name,...,train_type_code,tz_num,wz_num,yb_num,yp_ex,yp_info,yw_num,yz_num,ze_num,zy_num
0,05:31,Y,29,20300303,0,正常车次，不受控,0,0705,07,南京,...,2,--,322,--,10401030,2qI5iP6NO089WO62KUeJ%2BOiyKm6%2BaZ0u1opzaz%2B%...,91,528,--,--
1,05:38,Y,29,20300303,0,正常车次，不受控,0,0705,07,南京,...,2,--,无,--,10401030,EI4Ip%2B1i80plUz4ndoOA1KLaJd3y%2BvZHhyCGR%2FkB...,16,295,--,--
2,06:41,Y,29,20300303,0,正常车次，不受控,0,1707,19,成都东,...,2,--,无,--,O0M0O0,oJdpd6vDwCn5U6wHPQSaWRtfF89ZPh1AYOxmtZnoo23V75HW,--,--,75,11
3,06:40,Y,29,20300303,0,正常车次，不受控,0,2003,23,西安北,...,2,--,--,--,O0M090,MLRRkOrwXno5PHoT6uzVH%2FnR6nvkqEHpEyYDtkLGTpMQ...,--,--,707,160
4,06:50,Y,29,20300303,0,正常车次，不受控,0,0705,07,南京,...,3,--,无,--,O0O0M0,8Dc3rX%2BHIH2mumyslPLhh4jVHmGEwWxtNdCJExXMmZa3...,--,--,522,50


In [389]:
df.columns

Index([u'arrive_time', u'canWebBuy', u'control_day', u'control_train_day',
       u'controlled_train_flag', u'controlled_train_message',
       u'day_difference', u'end_city_code', u'end_province_code',
       u'end_station_name', u'end_station_telecode', u'from_station_name',
       u'from_station_no', u'from_station_telecode', u'gg_num', u'gr_num',
       u'is_support_card', u'lishi', u'lishiValue', u'location_code', u'note',
       u'qt_num', u'rw_num', u'rz_num', u'sale_time', u'seat_feature',
       u'seat_types', u'start_city_code', u'start_province_code',
       u'start_station_name', u'start_station_telecode', u'start_time',
       u'start_train_date', u'station_train_code', u'swz_num',
       u'to_station_name', u'to_station_no', u'to_station_telecode',
       u'train_class_name', u'train_no', u'train_seat_feature',
       u'train_type_code', u'tz_num', u'wz_num', u'yb_num', u'yp_ex',
       u'yp_info', u'yw_num', u'yz_num', u'ze_num', u'zy_num'],
      dtype='object')

In [392]:
df_use = df[[u'train_no',
            u'station_train_code',
             u'from_station_name',
             u'to_station_name',
             u'start_time',
             u'swz_num',
             u'tz_num',
             u'zy_num',
             u'ze_num',
             u'wz_num',
             ]]

In [393]:
df_use = df_use[df_use[u'station_train_code'].str.startswith('G') | \
       df_use[u'station_train_code'].str.startswith('D')]

In [394]:
df_use.head()

,train_no,station_train_code,from_station_name,to_station_name,start_time,swz_num,tz_num,zy_num,ze_num,wz_num
2,5l0000D63630,D636,上海虹桥,苏州,06:09,--,--,11,75,无
3,5l000G191200,G1912,上海虹桥,苏州北,06:17,23,--,160,707,--
4,55000G703240,G7032,上海,苏州,06:18,--,--,50,522,无
5,5l000G180200,G1802,上海虹桥,苏州北,06:22,无,--,17,448,--
6,55000G7076A0,G7076,上海,苏州,06:23,--,--,3,63,无


In [395]:
df_use.replace('--',0,inplace=True) # 注意：--是指无此席位，并非指无余票

In [396]:
df_use.replace(u'无',0,inplace=True)

In [397]:
# df_use.reset_index(inplace=True)
df_use.head()

,train_no,station_train_code,from_station_name,to_station_name,start_time,swz_num,tz_num,zy_num,ze_num,wz_num
2,5l0000D63630,D636,上海虹桥,苏州,06:09,0,0,11,75,0
3,5l000G191200,G1912,上海虹桥,苏州北,06:17,23,0,160,707,0
4,55000G703240,G7032,上海,苏州,06:18,0,0,50,522,0
5,5l000G180200,G1802,上海虹桥,苏州北,06:22,0,0,17,448,0
6,55000G7076A0,G7076,上海,苏州,06:23,0,0,3,63,0


In [398]:
df_use.to_excel('./data/ht2016-12-10-SH-NJ.xlsx',sheet_name='sheet1')

## 1.3一个完整代码

In [44]:
import requests
import pandas as pd
import json
#import arrow
import warnings
warnings.filterwarnings("ignore")

def get_pd(para):
    para = para
    r = requests.get('https://kyfw.12306.cn/otn/lcxxcx/query?purpose_codes=ADULT&' + \
                     para, verify=False).text
    return r

def to_csv(js):
    js2 = json.loads(js)
    data = js2['data']['datas']
    df = pd.DataFrame(data)
    df_use = df[[u'train_no',
            u'station_train_code',
                 u'from_station_name',
                 u'to_station_name',
                 u'start_time',
                 u'swz_num',
                 u'tz_num',
                 u'zy_num',
                 u'ze_num',
                 u'wz_num',
                 ]]
    df_use = df_use[df_use[u'station_train_code'].str.startswith('G') | \
    df_use[u'station_train_code'].str.startswith('D')]
    df_use.replace('--',0,inplace=True)
    df_use.replace(u'无',0,inplace=True)
#     tm = arrow.now().format('HH:mm:ss')
#     df_use['record_time'] = tm
    df_use.to_csv('./data/ht2016-12-10-SH-NJ.csv',index=False,encoding='utf-8')
if __name__ == '__main__':
    para = 'queryDate=2016-12-29&from_station=SHH&to_station=NJH'
    js = get_pd(para)
    to_csv(js)
    print('finished')

finished


In [ ]:
# 多sheet录入
writer = pd.ExcelWriter('./data/ht2016-12-10-SH-NJ.xlsx') 
df_use.to_excel(writer,'Sheet1')
df.to_excel(writer,'Sheet2') 
writer.save()

#  2. 类-面向对象

In [ ]:
hu_ning = ['SHH','KNH','SZH','WXH','CZH','DYH','ZJH','NJH']
ning_hu = ['NJH','ZJH','DYH','CZH','WXH','SZH','KNH','SHH']

In [37]:
hu_ning = ['SHH','KNH','SZH','WXH','CZH','DYH','ZJH','NJH']
# station_ls_comb = []
for i in range(0,len(hu_ning)-1):
    ls = [hu_ning[i],hu_ning[i+1]]
#     station_ls_comb.append(ls)
    print ls
# print station_ls_comb

['SHH', 'KNH']
['KNH', 'SZH']
['SZH', 'WXH']
['WXH', 'CZH']
['CZH', 'DYH']
['DYH', 'ZJH']
['ZJH', 'NJH']


## 2.1进行站点之间组合

In [399]:
from itertools import combinations
hu_ning = ['SHH','KNH','SZH','WXH','CZH','DYH','ZJH','NJH']
com = combinations(hu_ning,2)
for i in com:
    print i

('SHH', 'KNH')
('SHH', 'SZH')
('SHH', 'WXH')
('SHH', 'CZH')
('SHH', 'DYH')
('SHH', 'ZJH')
('SHH', 'NJH')
('KNH', 'SZH')
('KNH', 'WXH')
('KNH', 'CZH')
('KNH', 'DYH')
('KNH', 'ZJH')
('KNH', 'NJH')
('SZH', 'WXH')
('SZH', 'CZH')
('SZH', 'DYH')
('SZH', 'ZJH')
('SZH', 'NJH')
('WXH', 'CZH')
('WXH', 'DYH')
('WXH', 'ZJH')
('WXH', 'NJH')
('CZH', 'DYH')
('CZH', 'ZJH')
('CZH', 'NJH')
('DYH', 'ZJH')
('DYH', 'NJH')
('ZJH', 'NJH')


## 2.2抓取特定时间所有站点数据-运行一次

In [43]:
import requests
import pandas as pd
import json
import arrow
from itertools import combinations
import warnings
warnings.filterwarnings("ignore")

class HighSpeed(object):
    def __init__(self,date,from_station,to_station):
        self.date = date
        self.from_station = from_station
        self.to_station = to_station

    def get_pd(self):
        para = 'queryDate=' + self.date + '&from_station=' + self.from_station \
               +'&to_station=' + self.to_station
        r = requests.get('https://kyfw.12306.cn/otn/lcxxcx/query?purpose_codes=ADULT&' + \
                         para, verify=False).text
        return r

    def to_csv(self):
        js = self.get_pd()
        js2 = json.loads(js)
        data = js2['data']['datas']
        df = pd.DataFrame(data)
        df_use = df[[u'train_no',
                u'station_train_code',
                     u'from_station_name',
                     u'to_station_name',
                     u'start_time',
                     u'swz_num',
                     u'tz_num',
                     u'zy_num',
                     u'ze_num',
                     u'wz_num',
                     ]]
        df_use = df_use[df_use[u'station_train_code'].str.startswith('G') | \
        df_use[u'station_train_code'].str.startswith('D')]
        df_use.replace('--',0,inplace=True)
        df_use.replace(u'无',0,inplace=True)    
        
        tm = arrow.now().format('HH:mm')
        df_use['record_time'] = tm
        
#         df_use = df_choose[df_choose['record_time'] == df_choose['start_time']]
      
        df_use.to_csv('./data/' + self.date + '.csv',
                       mode = 'a', index = False, 
                      column = False,
                      encoding='utf-8')
         
if __name__ == '__main__':
    date = '2016-12-29'    
    hu_ning = ['SHH','KNH','SZH','WXH','CZH','DYH','ZJH','NJH']
    com = combinations(hu_ning,2)
    for ls in com:
        hspeed = HighSpeed(date,ls[0],ls[1])
        hspeed.to_csv()
    print('finished')

finished


# 3.按时间自动抓取余票数据

## 3.1 时间库与流程库

#### 流程控制库

In [ ]:
import schedule
import time

def job():
    print("I'm working...")

schedule.every(1).minutes.do(job)
# schedule.every().hour.do(job)
# schedule.every().day.at("14:09").do(job)
# schedule.every().monday.do(job)
# schedule.every().wednesday.at("13:15").do(job)

while True:
    schedule.run_pending()
    time.sleep(1)

I'm working...
I'm working...


#### 时间转换库

In [26]:
import arrow
tm = arrow.now()
print tm.format('HH:mm:ss')
    
tm_debefore30m = tm.replace(minutes=-30)
print tm_debefore30m

14:59:06
2016-12-15T14:29:06.772000+08:00


In [8]:
tm2_debefore30m = tm.replace(minutes=-30).format('HH:mm')
print tm2_debefore30m

13:46


In [43]:
date = arrow.now().format('YYYY-MM-DD')
date

u'2016-12-15'

## 3.2 每间隔一分钟抓取一次，所有站点数据-运行一次

In [ ]:
import requests
import pandas as pd
import json
import schedule
import time
import arrow
from itertools import combinations
import warnings
warnings.filterwarnings("ignore")


class HighSpeed(object):
    def __init__(self,date,from_station,to_station):
        self.date = date
        self.from_station = from_station
        self.to_station = to_station

    def get_pd(self):
        para = 'queryDate=' + self.date + '&from_station=' + self.from_station \
               +'&to_station=' + self.to_station
        r = requests.get('https://kyfw.12306.cn/otn/lcxxcx/query?purpose_codes=ADULT&' + \
                         para, verify=False).text
        return r

    def to_csv(self):
        js = self.get_pd()
        js2 = json.loads(js)
        data = js2['data']['datas']
        df = pd.DataFrame(data)
        df_use = df[[u'train_no',
                u'station_train_code',
                     u'from_station_name',
                     u'to_station_name',
                     u'start_time',
                     u'swz_num',
                     u'tz_num',
                     u'zy_num',
                     u'ze_num',
                     u'wz_num',
                     ]]
        df_use = df_use[df_use[u'station_train_code'].str.startswith('G') | \
        df_use[u'station_train_code'].str.startswith('D')]
        df_use.replace('--',0,inplace=True)
        df_use.replace(u'无',0,inplace=True)    
        
        tm = arrow.now()
        df_use['record_date'] = tm.format('YYYY-MM-DD')
        df_use['record_time'] = tm.format('HH:mm')
#         df_use = df_choose[df_choose['record_time'] == df_choose['start_time']]
      
        df_use.to_csv('./data/' + self.date + '.csv',
                      mode = 'a', index = False, 
                      header = False,encoding='utf-8')

    
        
if __name__ == '__main__':
       
    def process_hspeed():
        date = arrow.now().format('YYYY-MM-DD')    
        
        hu_ning = ['SHH','KNH','SZH','WXH','CZH','DYH','ZJH','NJH']
        com = combinations(hu_ning,2)
        for ls_hu_ning in com:
            hspeed = HighSpeed(date,ls_hu_ning[0],ls_hu_ning[1])
            hspeed.to_csv()
        
        ning_hu = ['NJH','ZJH','DYH','CZH','WXH','SZH','KNH','SHH']
        com = combinations(ning_hu,2)
        for ls_ning_hu in com:
            hspeed = HighSpeed(date,ls_ning_hu[0],ls_ning_hu[1])
            hspeed.to_csv()
            

    schedule.every(1).minutes.do(process_hspeed)
    # schedule.every().hour.do(job)
    # schedule.every().day.at("14:09").do(job)
    # schedule.every().monday.do(job)
    # schedule.every().wednesday.at("13:15").do(job)

    while True:
        schedule.run_pending()
        time.sleep(1)        

# 4.判断可用数据

In [605]:
import pandas as pd
import arrow
df = pd.read_csv('./data/NingHu-2016-12-17.csv',encoding='utf-8')
df.head()

,G7029,南京,镇江,05:49,0,0.1,97,996,0.2,2016-12-17,00:00
0,G7033,南京,镇江,06:06,0,0,48,457,168,2016-12-17,00:00
1,G7101,南京,镇江,06:11,0,0,47,494,166,2016-12-17,00:00
2,G7035,南京,镇江,06:31,0,0,49,489,168,2016-12-17,00:00
3,G7581,南京南,镇江,06:45,10,0,25,463,146,2016-12-17,00:00
4,G7505,南京南,镇江南,07:13,26,0,105,545,264,2016-12-17,00:00


In [606]:
df.columns = [u'station_train_code',
                     u'from_station_name',
                     u'to_station_name',
                     u'start_time',
                     u'swz_num',
                     u'tz_num',
                     u'zy_num',
                     u'ze_num',
                     u'wz_num',
              u'record_date',
              u'record_time',]
df.head()

,station_train_code,from_station_name,to_station_name,start_time,swz_num,tz_num,zy_num,ze_num,wz_num,record_date,record_time
0,G7033,南京,镇江,06:06,0,0,48,457,168,2016-12-17,00:00
1,G7101,南京,镇江,06:11,0,0,47,494,166,2016-12-17,00:00
2,G7035,南京,镇江,06:31,0,0,49,489,168,2016-12-17,00:00
3,G7581,南京南,镇江,06:45,10,0,25,463,146,2016-12-17,00:00
4,G7505,南京南,镇江南,07:13,26,0,105,545,264,2016-12-17,00:00


In [607]:
time_choose = arrow.now()
print time_choose
time_before_30m = time_choose.replace(minutes=-30).format('HH:mm')
print time_before_30m

2016-12-18T18:14:44.114648+08:00
17:44


In [608]:
df[df['start_time'] == '06:06'].head(2)

,station_train_code,from_station_name,to_station_name,start_time,swz_num,tz_num,zy_num,ze_num,wz_num,record_date,record_time
0,G7033,南京,镇江,06:06,0,0,48,457,168,2016-12-17,00:00
637,G7033,南京,镇江,06:06,0,0,48,465,168,2016-12-17,00:01


In [609]:
df[(df['start_time'] == '08:35') & (df['record_time'] == '08:20')]

,station_train_code,from_station_name,to_station_name,start_time,swz_num,tz_num,zy_num,ze_num,wz_num,record_date,record_time
296760,D3135,镇江,丹阳,08:35,0,0,99,268,121,2016-12-17,08:20
296795,D2281,丹阳,常州,08:35,0,0,91,388,123,2016-12-17,08:20
297076,G7039,苏州,昆山南,08:35,0,0,31,29,122,2016-12-17,08:20


## 4.1 筛选出发车时间前30分钟的余票信息

> 上海——南京，最早发车时间为06:02，最后一个区间是	22:43  G7128  镇江至南京，<br>
> 南京——上海，最早发车时间为03:30，最后一个区间是	22:43  G7128  镇江至南京，

In [610]:
df2 = df[(df['start_time'].str.contains("0[6-9]:|1[0-9]:|2[0-3]:"))]
df2['start_time'].sort_values(ascending=True).head()

0         06:06
170704    06:06
82801     06:06
123569    06:06
217196    06:06
Name: start_time, dtype: object

In [611]:
df2.head()

,station_train_code,from_station_name,to_station_name,start_time,swz_num,tz_num,zy_num,ze_num,wz_num,record_date,record_time
0,G7033,南京,镇江,06:06,0,0,48,457,168,2016-12-17,00:00
1,G7101,南京,镇江,06:11,0,0,47,494,166,2016-12-17,00:00
2,G7035,南京,镇江,06:31,0,0,49,489,168,2016-12-17,00:00
3,G7581,南京南,镇江,06:45,10,0,25,463,146,2016-12-17,00:00
4,G7505,南京南,镇江南,07:13,26,0,105,545,264,2016-12-17,00:00


In [16]:
df2['start_time'].head()

0    06:23
1    06:32
2    06:38
3    06:50
4    07:15
Name: start_time, dtype: object

In [410]:
arrow.get('08:05', 'HH:mm').replace(minutes=-30).format('HH:mm')

'07:35'

In [612]:
df2['start_time'].apply(lambda x: arrow.get(x, 'HH:mm').replace(minutes=-30).format('HH:mm')).head()

0    05:36
1    05:41
2    06:01
3    06:15
4    06:43
Name: start_time, dtype: object

In [613]:
df2['start_time_before30m'] = df2['start_time'].apply(lambda x: arrow.get(x, 'HH:mm')\
                                                      .replace(minutes=-30).format('HH:mm'))
df2.head()

,station_train_code,from_station_name,to_station_name,start_time,swz_num,tz_num,zy_num,ze_num,wz_num,record_date,record_time,start_time_before30m
0,G7033,南京,镇江,06:06,0,0,48,457,168,2016-12-17,00:00,05:36
1,G7101,南京,镇江,06:11,0,0,47,494,166,2016-12-17,00:00,05:41
2,G7035,南京,镇江,06:31,0,0,49,489,168,2016-12-17,00:00,06:01
3,G7581,南京南,镇江,06:45,10,0,25,463,146,2016-12-17,00:00,06:15
4,G7505,南京南,镇江南,07:13,26,0,105,545,264,2016-12-17,00:00,06:43


In [614]:
df2[df2['start_time_before30m'] == df2['record_time']].head(5)

,station_train_code,from_station_name,to_station_name,start_time,swz_num,tz_num,zy_num,ze_num,wz_num,record_date,record_time,start_time_before30m
200643,G7033,南京,镇江,06:06,0,0,48,466,168,2016-12-17,05:36,05:36
203192,G7101,南京,镇江,06:11,0,0,48,493,166,2016-12-17,05:41,05:41
203301,G7029,镇江,丹阳,06:11,0,0,96,914,0,2016-12-17,05:41,05:41
212260,G7029,丹阳,常州,06:26,0,0,94,897,0,2016-12-17,05:56,05:56
215292,G7035,南京,镇江,06:31,0,0,50,489,168,2016-12-17,06:01,06:01


#### df3为可用数据

In [615]:
df3 = df2[df2['start_time_before30m'] == df2['record_time']]
print len(df3)

586


In [616]:
df3[df3['station_train_code'] == 'G7505']

,station_train_code,from_station_name,to_station_name,start_time,swz_num,tz_num,zy_num,ze_num,wz_num,record_date,record_time,start_time_before30m
265186,G7505,常州北,无锡东,07:53,24,0,103,420,258,2016-12-17,07:23,07:23
275684,G7505,无锡东,苏州北,08:12,24,0,96,273,256,2016-12-17,07:42,07:42
283004,G7505,苏州北,昆山南,08:25,23,0,86,166,247,2016-12-17,07:55,07:55
290760,G7505,昆山南,上海虹桥,08:38,23,0,82,4,248,2016-12-17,08:08,08:08


In [617]:
df3['from_to'] = np.nan
df3['from_to'] = df3['from_station_name'] + ',' +df3['to_station_name']
df3['from_to'] = df3['from_to'].str.split(',')
df3.head(2)

,station_train_code,from_station_name,to_station_name,start_time,swz_num,tz_num,zy_num,ze_num,wz_num,record_date,record_time,start_time_before30m,from_to
200643,G7033,南京,镇江,06:06,0,0,48,466,168,2016-12-17,05:36,05:36,"[南京, 镇江]"
203192,G7101,南京,镇江,06:11,0,0,48,493,166,2016-12-17,05:41,05:41,"[南京, 镇江]"


#### bak

In [618]:
print df3['from_to'][200643]

[u'\u5357\u4eac', u'\u9547\u6c5f']


In [619]:
df3['from_to'][200643] == [u'南京', u'镇江']

True

In [548]:
print type(df3.iloc[1,-1].decode('utf-8'))

<type 'unicode'>


In [549]:
print df3.iloc[1,-1].decode('utf-8')

南京,镇江


## 2.2 汇总前一区间与下一区间的余票，并计算站点的上下客情况。

>step1:添加一列，统计余票总数（如果有可能，可去获取上车多少人？）;<br>
>step2:前一个数据减后一个数据，即可获得中间站的上下客情况。

>中间站即为下一条数据的from_station_name

# 二、获取各车次区间情况

# 1. 区间站信息获取

## 1.1 获取沪宁-宁沪线车次

https://kyfw.12306.cn/otn/czxx/queryByTrainNo?train_no=5l000G191200&from_station_telecode=SHH&to_station_telecode=NJH&depart_date=2016-12-23

In [6]:
import requests
import pandas as pd
import json
#import arrow
import warnings
warnings.filterwarnings("ignore")

def get_pd(para):
    para = para
    r = requests.get('https://kyfw.12306.cn/otn/lcxxcx/query?purpose_codes=ADULT&' + \
                     para, verify=False).text
    return r

def to_csv(js):
    js2 = json.loads(js)
    data = js2['data']['datas']
    df = pd.DataFrame(data)
    df_use = df[[u'train_no',
            u'station_train_code',
                 u'from_station_name',
                 u'to_station_name',
                 u'start_time',
                 u'swz_num',
                 u'tz_num',
                 u'zy_num',
                 u'ze_num',
                 u'wz_num',
                 ]]
    df_use = df_use[df_use[u'station_train_code'].str.startswith('G') | \
    df_use[u'station_train_code'].str.startswith('D')]
    df_use.replace('--',0,inplace=True)
    df_use.replace(u'无',0,inplace=True)
#     tm = arrow.now().format('HH:mm:ss')
#     df_use['record_time'] = tm
#     df_use[u'train_no'].to_csv('./data/SH-NJ.csv',index=False,encoding='utf-8')
    df_use[u'train_no'].to_csv('./data/NJ-SH.csv',index=False,encoding='utf-8')
    
if __name__ == '__main__':
#     para = 'queryDate=2016-12-29&from_station=SHH&to_station=NJH'
    para = 'queryDate=2016-12-29&from_station=NJH&to_station=SHH'
    js = get_pd(para)
    to_csv(js)
    print('finished')

finished


## 1.2 获取各车次，站点信息

In [266]:
import requests
import pandas as pd
import json
#import arrow
import warnings
warnings.filterwarnings("ignore")


def get_pd(para):
    para = para
    r = requests.get('https://kyfw.12306.cn/otn/czxx/queryByTrainNo?train_no='+para+\
                     '&from_station_telecode=SHH&to_station_telecode=NJH&depart_date=2016-12-23',
                     verify=False).text
    return r

def to_csv(para,js):
    js2 = json.loads(js)
    data = js2['data']['data']
    df = pd.DataFrame(data)
    try:
        df = df[["station_name",]]
        df_use = pd.DataFrame({'train_no':[''],'station_name':['']})
        df_use['train_no'] = para.strip()
        df_use['station_name'] = df_use['station_name'].apply(lambda x: df["station_name"].values)
    #     df_use['start_time'] = df_use['start_time'].apply(lambda x: df["start_time"].values)
    #     df_use = df_use[df_use[u'station_train_code'].str.startswith('G') | \
    #     df_use[u'station_train_code'].str.startswith('D')]
    #     df_use.replace('--',0,inplace=True)
    #     df_use.replace(u'无',0,inplace=True)
    #     tm = arrow.now().format('HH:mm:ss')
    #     df_use['record_time'] = tm
        df_use.to_csv('./data/SH-NJ-station.csv',mode = 'a',index=False,header=False,encoding='utf-8')
    except:
        pass
if __name__ == '__main__':
#     para = '55000G707270'
#     js = get_pd(para)
#     to_csv(para,js)
#     print('finished')
    station_no_file = open('./data/SH-NJ.csv','r')
    station_no_data = station_no_file.readlines()
    for station_no in station_no_data:
#         print station_no.strip()
#         para = '55000G707270'
        js = get_pd(station_no.strip())
        to_csv(station_no.strip(),js)
    print('finished')

finished


#### 读取信息

In [620]:
df = pd.read_csv('./data/SH-NJ-station.csv',encoding='utf-8')
df.tail()

,"[上海, 无锡, 常州, 丹阳, 镇江, 宝华山, 南京南, 马鞍山东, 当涂东, 芜湖, 弋江, 铜陵, 池州, 安庆]",55000G707270
196,"[上海, 苏州, 南京, 北京南]",550000D31490
197,"[上海虹桥, 昆山南, 苏州, 无锡, 常州, 镇江, 南京]",5l000G7128A0
198,"[深圳北, 惠州南, 惠东, 汕尾, 潮阳, 潮汕, 诏安, 漳浦, 漳州, 厦门北, 泉州...",6i000D228207
199,"[苍南, 鳌江, 温州南, 青田, 丽水, 缙云西, 永康南, 武义北, 义乌, 杭州东, ...",5j000G735010
200,"[上海, 苏州, 南京, 徐州, 郑州东, 洛阳龙门, 三门峡南, 华山北, 渭南北, 西安北]",550000D30661


In [621]:
df.columns=['station_name','train_no']

#### 添加列，进行组合

In [622]:
print df['station_name'][0]

[上海虹桥, 无锡, 常州, 镇江, 南京南, 合肥南, 六安, 金寨, 麻城北, 汉口, 荆州, 宜昌东, 建始, 恩施, 利川, 石柱县, 涪陵北, 重庆北, 潼南, 遂宁, 成都东]


In [623]:
df['station_name'][0]

u'[\u4e0a\u6d77\u8679\u6865, \u65e0\u9521, \u5e38\u5dde, \u9547\u6c5f, \u5357\u4eac\u5357, \u5408\u80a5\u5357, \u516d\u5b89, \u91d1\u5be8, \u9ebb\u57ce\u5317, \u6c49\u53e3, \u8346\u5dde, \u5b9c\u660c\u4e1c, \u5efa\u59cb, \u6069\u65bd, \u5229\u5ddd, \u77f3\u67f1\u53bf, \u6daa\u9675\u5317, \u91cd\u5e86\u5317, \u6f7c\u5357, \u9042\u5b81, \u6210\u90fd\u4e1c]'

In [569]:
df['station_name'] = df['station_name'].apply(lambda x: \
                                              df['station_name'].str.replace(u'[',u'').str.replace(u']',u'')\
                                             .str.replace(' ','').str.split(','))

In [570]:
type(df['station_name'][0])

list

In [571]:
len(df['station_name'][0])

21

[上海虹桥, 无锡, 常州, 镇江, 南京南, 合肥南, 六安, 金寨, 麻城北, 汉口, 荆州, 宜昌东, 建始, 恩施, 利川, 石柱县, 涪陵北, 重庆北, 潼南, 遂宁, 成都东]

In [624]:
dm = df.copy()

### 将2数进行组合（zip与map函数、join方法的用法）

In [472]:
seq1 = ['one0', 'one1', 'one2']
seq2 = ['two0', 'two1', 'two2']
zip(seq1, seq2)

[('one0', 'two0'), ('one1', 'two1'), ('one2', 'two2')]

In [515]:
seq = ['one0', 'one1', 'one2', 'two0', 'two1', 'two2']
com= map(list,zip(seq[:-1], seq[1:]))
com

[['one0', 'one1'],
 ['one1', 'one2'],
 ['one2', 'two0'],
 ['two0', 'two1'],
 ['two1', 'two2']]

In [517]:
','.join(com[0])

'one0,one1'

In [ ]:
my_df['B'] = my_df.A.apply(lambda x: list(map(list, zip(x[:-1], x[1:]))))
my_df

In [625]:
dm.head()

,station_name,train_no
0,"[上海虹桥, 无锡, 常州, 镇江, 南京南, 合肥南, 六安, 金寨, 麻城北, 汉口, ...",5l0000D35270
1,"[上海虹桥, 苏州, 无锡, 常州, 镇江, 南京南, 全椒, 肥东, 合肥南, 六安, 金...",5l0000D63630
2,"[上海虹桥, 无锡东, 常州北, 镇江南, 南京南, 蚌埠南, 徐州东, 枣庄, 曲阜东, ...",5l0000G45630
3,"[上海虹桥, 苏州北, 常州北, 镇江南, 南京南, 蚌埠南, 徐州东, 砀山南, 商丘, ...",5l000G191200
4,"[上海, 昆山南, 苏州, 无锡, 常州, 丹阳, 镇江, 南京]",55000G703240


In [584]:
[u'上海虹桥',u'无锡'] in dm['station_interval'][0]

True

In [ ]:
dm['station_interval'] = dm['station_name'].apply(lambda x: map(list,zip(x[:-1], x[1:])))
dm.head()

In [588]:
dm['station_interval'][0][0]

[u'\u4e0a\u6d77\u8679\u6865', u'\u65e0\u9521']

In [486]:
u'上海虹桥, 无锡' in df['station_interval'][0][0]

False

#### 列表推导

In [446]:
df3.head(2)

,station_train_code,from_station_name,to_station_name,start_time,swz_num,tz_num,zy_num,ze_num,wz_num,record_date,record_time,start_time_before30m,from_to
200643,G7033,南京,镇江,06:06,0,0,48,466,168,2016-12-17,05:36,05:36,"南京,镇江"
203192,G7101,南京,镇江,06:11,0,0,48,493,166,2016-12-17,05:41,05:41,"南京,镇江"


In [453]:
df.station_interval[0]

[(u'\u4e0a\u6d77\u8679\u6865', u'\u65e0\u9521'),
 (u'\u65e0\u9521', u'\u5e38\u5dde'),
 (u'\u5e38\u5dde', u'\u9547\u6c5f'),
 (u'\u9547\u6c5f', u'\u5357\u4eac\u5357'),
 (u'\u5357\u4eac\u5357', u'\u5408\u80a5\u5357'),
 (u'\u5408\u80a5\u5357', u'\u516d\u5b89'),
 (u'\u516d\u5b89', u'\u91d1\u5be8'),
 (u'\u91d1\u5be8', u'\u9ebb\u57ce\u5317'),
 (u'\u9ebb\u57ce\u5317', u'\u6c49\u53e3'),
 (u'\u6c49\u53e3', u'\u8346\u5dde'),
 (u'\u8346\u5dde', u'\u5b9c\u660c\u4e1c'),
 (u'\u5b9c\u660c\u4e1c', u'\u5efa\u59cb'),
 (u'\u5efa\u59cb', u'\u6069\u65bd'),
 (u'\u6069\u65bd', u'\u5229\u5ddd'),
 (u'\u5229\u5ddd', u'\u77f3\u67f1\u53bf'),
 (u'\u77f3\u67f1\u53bf', u'\u6daa\u9675\u5317'),
 (u'\u6daa\u9675\u5317', u'\u91cd\u5e86\u5317'),
 (u'\u91cd\u5e86\u5317', u'\u6f7c\u5357'),
 (u'\u6f7c\u5357', u'\u9042\u5b81'),
 (u'\u9042\u5b81', u'\u6210\u90fd\u4e1c')]